<a href="https://colab.research.google.com/github/mansoorshakeel0-stack/Project/blob/main/Credit%20Scoring%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score


In [2]:
np.random.seed(42)
n = 1000

data = pd.DataFrame({
    'income': np.random.normal(50000, 15000, n),
    'debt': np.random.normal(15000, 8000, n),
    'age': np.random.randint(21, 65, n),
    'num_late_payments': np.random.poisson(2, n),
    'loan_amount': np.random.normal(10000, 5000, n),
})


In [3]:
data['creditworthy'] = (
    (data['income'] > 40000) &
    (data['debt'] < 20000) &
    (data['num_late_payments'] < 4)
).astype(int)


In [4]:
data['debt_to_income'] = data['debt'] / data['income']
data['loan_to_income'] = data['loan_amount'] / data['income']

In [5]:
X = data[['income', 'debt', 'age', 'num_late_payments', 'loan_amount',
          'debt_to_income', 'loan_to_income']]
y = data['creditworthy']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42)
}


In [9]:
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]

    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, y_prob):.3f}")



=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       149
           1       0.83      0.81      0.82       151

    accuracy                           0.82       300
   macro avg       0.82      0.82      0.82       300
weighted avg       0.82      0.82      0.82       300

ROC-AUC Score: 0.911

=== Decision Tree ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       149
           1       1.00      1.00      1.00       151

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

ROC-AUC Score: 1.000

=== Random Forest ===
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       149
           1       1.00      0.99      1.00       151

    accuracy                           1.00       300
   macro av

In [11]:
results = []
for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    results.append({
        "Model": name,
        "ROC-AUC": roc_auc_score(y_test, y_prob),
        "F1-Score": classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
    })

In [12]:
summary = pd.DataFrame(results)
print("\nModel Comparison Summary:\n", summary)


Model Comparison Summary:
                  Model   ROC-AUC  F1-Score
0  Logistic Regression  0.911418  0.819984
1        Decision Tree  1.000000  1.000000
2        Random Forest  1.000000  0.996667
